# Heisenberg Model Simulation using Trotterization

In [5]:
import json
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

from functools import partial

In [2]:
dev = qml.device("default.qubit", wires=4)

# Hamiltonian for a Heisenberg chain with 4 spins
def create_hamiltonian(params):
    couplings = [-params[-1]]
    ops = [qml.PauliX(3)]
    for i in range(3):
        couplings = [-params[-1]] + couplings
        ops = [qml.PauliX(i)] + ops        
    for i in range(4):
        couplings = [-params[-2]] + couplings
        ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops
    for i in range(4):
        couplings = [-params[-3]] + couplings
        ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops
    for i in range(4):
        couplings = [-params[0]] + couplings
        ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    
    return qml.Hamiltonian(couplings,ops)

# Circuit for the time evolution of the Heisenberg chain
@partial(qml.transforms.decompose, gate_set={qml.CNOT, qml.RX, qml.RY, qml.RZ})
@qml.qnode(dev)
def evolve(params, time, depth):
    qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))

In [3]:
print(create_hamiltonian([1,3,2,0.3]))

-1 * (X(3) @ X(0)) + -1 * (X(2) @ X(3)) + -1 * (X(1) @ X(2)) + -1 * (X(0) @ X(1)) + -3 * (Y(3) @ Y(0)) + -3 * (Y(2) @ Y(3)) + -3 * (Y(1) @ Y(2)) + -3 * (Y(0) @ Y(1)) + -2 * (Z(3) @ Z(0)) + -2 * (Z(2) @ Z(3)) + -2 * (Z(1) @ Z(2)) + -2 * (Z(0) @ Z(1)) + -0.3 * X(2) + -0.3 * X(1) + -0.3 * X(0) + -0.3 * X(3)


In [4]:
print(qml.draw(evolve)([1,3,2,0.3], 3, 2))

0: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╭●────────────╭●──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)
1: ───────────────────────────────│─────────────│───RZ(1.57)──RX(1.57)──RZ(1.57)────────────────────
2: ───────────────────────────────│─────────────│───RZ(1.57)──RX(1.57)──RZ(1.57)────────────────────
3: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╰X──RZ(-3.00)─╰X──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)

───RZ(1.57)────────────────────────────────────────────────────────────────────────────────
─────────────────────────────────────────────────────────────────────────────────────────╭X
────────────╭X──RZ(-3.00)─╭X──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)──RZ(1.57)─╰●
───RZ(1.57)─╰●────────────╰●──RZ(1.57)──RX(1.57)──RZ(1.57)──RX(1.57)───────────────────────

────────────────────────────────────────────────────────────────────────────╭X──RZ(-3.00)─╭X
───RZ(-3.00)─╭X──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)──RZ(1.57)─╰●────────────╰●
─────────────╰●──RZ(1.57)──RX(1.57)──RZ(

In [6]:
result = (evolve)([1,3,2,0.3], 3, 2)
print(result)

0.008660043937568938


<hr/>